In [44]:
import os
from ase.calculators.vasp import Vasp2
from ase.io import read
import pickle as pkl
import ase
import numpy as np
from ase.eos import EquationOfState
from ase.dft.bandgap import bandgap

In [2]:
def clean_up():
    os.system("rm ase-sort.dat C* D* E* I* K* O* P* vasp.out vasprun* W* X*")

In [3]:
clean_up()

# Volume Scan Functionality

In [68]:
def do_autotune_volume(system,calc,start_scan=0.95,end_scan=1.05,num_scan=10,exclude_type=None,only_type=None):
    # Load system and calculator
    if isinstance(system,ase.atoms.Atoms):
        print("System loaded.")
    else:
        print("System must be of type `ase.atoms.Atoms`. Got type `{}`.".format(type(system)))
        return
    if isinstance(calc,ase.calculators.vasp.Vasp2):
        system.set_calculator(calc)
        print("Calculator loaded and attached to system.")
    else:
        print("Calculator must be of type `ase.calculator.vasp.Vasp2`. Got type `{}`.".format(type(calc)))
        return
    
    # Scale volumes and collect energy & volume measurements
    ens = []
    vols = []
    start_cell = system.get_cell()
    
    print("Performing volume scan.")
    for i,x in enumerate(np.linspace(start_scan,end_scan,num_scan)):
        #print(i)
        system.set_cell(x*start_cell,scale_atoms=True)
        vols.append(system.get_volume())
        ens.append(system.get_potential_energy())
        
    # Fit to Equations of States
    print("Fitting data to equations of state.")
    eos_types = "sjeos taylor murnaghan birch birchmurnaghan pouriertarantola p3 antonschmidt".split()
    if exclude_type != None:
        _ = [eos_types.remove(i) for i in exclude_type]
    elif only_type != None:
        eos_types = only_type
        
    eos_fits = {}
    
    for typ in eos_types:
        eos = EquationOfState(vols,ens,eos=typ)
        v, e, B = eos.fit()
        eos_fits[typ] = {'volume':v,'energy':e,'buld_modulus':B}
        
    # Rescale initial cell to optimized volume
    print("Rescaling with optimized volume.")
    vol_avg = np.average([eos_fits[key]['volume'] for key in eos_fits.keys()])
    system.set_cell(start_cell,scale_atoms=True)
    
    print("Performing second relaxation.")
    system.get_potential_energy()
    
    clean_up()
    
    return system
    

In [37]:
calc.set(xc='pbe',setups=potcars[1])
atoms = do_autotune_volume(atoms,calc)
atoms.get_volume()


System loaded.
Calculator loaded and attached to system.
Performing volume scan.


KeyboardInterrupt: 

# Bandgap Functionality

In [57]:
def get_bandgap(system):
    gap,p1,p2 = bandgap(system.calc);
    return gap

In [58]:
get_bandgap(atoms)

Gap: 1.883 eV
Transition (v -> c):
  (s=0, k=0, n=7, [0.000, 0.000, 0.000]) -> (s=0, k=0, n=8, [0.000, 0.000, 0.000])


1.8830000000000005

# Setup System

This is the basic setup that will need to be done for every ValetJob before the volume autotune and bandgap calculation.

## Inputs

In [64]:
poscar = '../valet_workspace/GaN_804/POSCAR'
kpts = [4.,4.,4.]
incar = '../calculators/INCAR.scwf'
potcar_pkl = '../calculators/GaN_potcars.pkl' 

## Load Necessary Files

In [65]:
atoms = read(poscar)
potcars = pkl.load(open(potcar_pkl,'rb'))
calc = Vasp2()
calc.read_incar(incar)
calc.set(kpts=kpts,gamma=True) # Gamma must be True for vasp to work in all cases
atoms.set_calculator(calc)

# Perform Volume Scan

In [71]:
potcar = potcars[1] # Select POTCAR
calc.set(xc='pbe',setups=potcar)
atoms = do_autotune_volume(atoms,calc)

System loaded.
Calculator loaded and attached to system.
Performing volume scan.
Fitting data to equations of state.
Rescaling with optimized volume.
Performing second relaxation.


# Get Bandgap

In [72]:
gap = get_bandgap(atoms)
print(potcar,gap)

Gap: 1.805 eV
Transition (v -> c):
  (s=0, k=0, n=17, [0.000, 0.000, 0.000]) -> (s=0, k=0, n=18, [0.000, 0.000, 0.000])
{'base': 'recommended', 'Ga': '_d'} 1.8053999999999997
